<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/jc/3_make_errors/e4_make_errors_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install kiwipiepy
# !pip install nltk
# !pip install
# !pip install selenium

# #코랩에서 구글드라이버 연동
# !apt-get update
# !apt install chromium-chromedriver

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


**아이디어 기록**

1. Critical

2. Major
    - Mis-Translation
        - "ailen" -> 외계인으로 처리

# 0.Data Load

In [3]:
import pandas as pd
import numpy as np
import json
import os
import re
import copy
from tqdm.notebook import tqdm

if os.path.exists(r"G:\내 드라이브\KB인턴관련"):
    gdrive_path = r"G:\내 드라이브\KB인턴관련"
else:
    gdrive_path = "/content/drive/MyDrive/KB인턴관련"

In [4]:
df = pd.read_csv(os.path.join(gdrive_path, "preprocessing/fin.csv"), encoding="euc-kr", index_col=0)
df.head()

,category,lawName,ENG,KOR
0,헌법,개성공업지구 지원에 관한 법률,The purpose of this Act is to enhance inter-Ko...,이 법은 개성공업지구의 개발·운영의 지원 및 개성공업지구에 투자하거나 출입·체류하는...
2,헌법,개성공업지구 지원에 관한 법률,"The term ""Gaeseong Industrial Complex"" or ""GIC...","""개성공업지구""란 남한과 북한 사이의 합의에 따라서 북한의 개성지역 일대에서 개발·..."
3,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC developer"" means a South Korean ...","""개성공업지구 개발업자""란 「남북교류협력에 관한 법률」에 따른 협력사업의 승인을 받..."
4,헌법,개성공업지구 지원에 관한 법률,"The term ""GIC management organ"" means a corpor...","""개성공업지구 관리기관""이란 개성공업지구의 관리·운영을 위하여 북한의 「개성공업지구..."
5,헌법,개성공업지구 지원에 관한 법률,"The term ""local enterprise of the GIC"" means a...","""개성공업지구 현지기업""이란 남한주민이 「남북교류협력에 관한 법률」에 따른 협력사업..."


# 1.Data Fiiltering

In [5]:
keyword_idx = df.loc[df.lawName.str.contains(r"은행|금융|보험|예금|적금|대출|외환|통화|화폐|환율|예대")].index

In [6]:
tgk_idx = df.loc[df.category=="통화ㆍ국채ㆍ금융"].index

In [7]:
print(f"키워드: {len(keyword_idx)}\t통화/국채/금융: {len(tgk_idx)}")
print(f"키워드-통국금: {len(list(set(keyword_idx)-set(tgk_idx)))}")
print(f"통국금-키워드: {len(list(set(tgk_idx)-set(keyword_idx)))}")
print(f"전체:  {len(list(set(keyword_idx).union(set(tgk_idx))))}")

키워드: 16712	통화/국채/금융: 16004
키워드-통국금: 4681
통국금-키워드: 3973
전체:  20685


In [8]:
df_samp = df.loc[list(set(keyword_idx).union(set(tgk_idx)))].drop(index=[403163], axis=0)
# df_samp = df.copy(deep=True)

In [9]:
df_samp["curHyp"] = df_samp.KOR.values
df_samp["errors"] = [[] for _ in range(df_samp.shape[0])]
df_samp["tagLog"] = [dict() for _ in range(df_samp.shape[0])]
# df_samp.iloc[:, 2:4]

**주요 함수**

In [10]:
## 주요한 함수
def remake_df():
    result = df.loc[list(set(keyword_idx).union(set(tgk_idx)))]
    result["curHyp"] = result.KOR.values
    result["errors"] = [[] for _ in range(result.shape[0])]
    result["tag_log"] = [dict() for _ in range(result.shape[0])]
    return result.drop(index=[403163], axis=0)

def random_df(dfs, ratio, seed):
    np.random.seed(seed)
    return dfs.loc[np.random.choice(dfs.index.to_list(), int(dfs.shape[0]*ratio), False), :]

def random_index(srs, ratio, seed):
    np.random.seed(seed)
    return np.random.choice(srs.index, int(srs.shape[0]*ratio), False)

def copy_df(data, col=["errors", "tagLog"]):
    result = data.drop(columns=col, axis=1).copy(deep=True)
    # print(id(result), id(data))
    for c in col:
        result[c] = data[c].apply(lambda x: copy.deepcopy(x)).copy(deep=True)
    return result

def count_tag(tokens, tags=None):
    token_dict = dict()
    for token in tokens:
        if tags!=None and token.tag not in tags: continue
        try:
            token_dict[token.tag].append(token.form)
        except:
            token_dict[token.tag] = [token.form]

    return token_dict

def find_tokens(tokens, tag):
    result_tokens = []
    for token in tokens:
        if token.tag == tag:
            result_tokens.append((token.form, token.start, token.end))
    return result_tokens

def print_df(df, cols, idx=0, end=0):
    print(df.shape[0])
    if end==0:
        values = df.index[idx:]
    else:
        values = df.index[idx:end]
    for i in values:
        print(f"_________________ {i} _________________")
        for col in cols:
            print(df.loc[i, col])

# result.loc[i, "errors"].append({"severity":"major", "start_idx":start, "text":del_unit, "type":"omission"})
def input_errors(df, index, severity, start_idx, text, type):
    df.loc[index, "errors"].append({"severity":severity, "start_idx":start_idx, "text":text, "type":type})

def convert_df(df, diff_df):
    cdf = copy_df(df)
    for i in diff_df.index:
        cdf.loc[i, : ] = diff_df.loc[i, : ]
    return cdf

def find_severity(df, sv):
    df.errors.apply()

# 2.품사 태깅

In [58]:
from kiwipiepy import Kiwi

kiwi = Kiwi(num_workers=0, model_type='sbg')
kiwi_tokens = df_samp.KOR.apply(lambda x: kiwi.tokenize(x))
kiwi_tokens

547121    [(예술인, NNG, 0, 3), ([, SSO, 3, 1), (「, SSO, 4,...
547123    [(노무, NNG, 0, 2), (제공자, NNG, 2, 3), ([, SSO, 5...
547124    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547125    [(법, NNG, 0, 1), (제, XPN, 2, 1), (3, SN, 3, 1)...
547126    [(예술, NNG, 0, 2), (이, VCP, 2, 1), (ᆫ, ETM, 2, ...
                                ...                        
78005     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
78006     [(제, XPN, 0, 1), (1, SN, 1, 1), (항, NNG, 2, 1)...
78007     [(기획, NNG, 0, 2), (재정부, NNG, 2, 3), (장관, NNG, ...
406361    [(제, XPN, 0, 1), (3, SN, 1, 1), (항, NNG, 2, 1)...
406362    [(조합, NNG, 0, 2), (이, JKS, 2, 1), (제, XPN, 4, ...
Name: KOR, Length: 20684, dtype: object

In [59]:
count_tokens = kiwi_tokens.apply(count_tag)
count_tokens

547121    {'NNG': ['예술인', '항', '단서', '단기', '예술인', '이하', ...
547123    {'NNG': ['노무', '제공자', '항', '단서', '단기', '노무', '...
547124    {'NNG': ['법', '항', '대통령령', '사유', '해당', '경우', '...
547125    {'NNG': ['법', '항', '법', '항', '항', '준용', '경우', ...
547126    {'NNG': ['예술', '다음', '호의', '해당', '경우', '법', '항...
                                ...                        
78005     {'NNG': ['기획', '재정부', '장관', '국채', '발행', '관리', ...
78006     {'XPN': ['제'], 'SN': ['1'], 'NNG': ['항', '국채',...
78007     {'NNG': ['기획', '재정부', '장관', '국채', '시장', '정책', ...
406361    {'XPN': ['제', '제', '제', '제', '제'], 'SN': ['3',...
406362    {'NNG': ['조합', '항', '규정', '복지', '사업', '실시', '경...
Name: KOR, Length: 20684, dtype: object

In [60]:
nng_dict = dict()
for tokens in count_tokens.apply(lambda x: x["NNG"]):
    for t in tokens:
        try:
            nng_dict[t] += 1
        except:
            nng_dict[t] = 1
len(nng_dict.keys())

3703

In [61]:
nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, "NNP"))
nnp_info

547121    [(고용보험법, 5, 10)]
547123    [(고용보험법, 7, 12)]
547124                  []
547125                  []
547126                  []
                ...       
78005     [(한국은행, 56, 60)]
78006                   []
78007                   []
406361                  []
406362                  []
Name: KOR, Length: 20684, dtype: object

# 3.Error 생성

In [87]:
dindex_dict = {"omission":[], "addition":[], "mis-translation":[], "over-translation":[], "under-translation":[]}

## 3.1.Omission

### Omission - Critical

In [88]:
cr_om_index = random_index(df_samp.curHyp, 0.02, 10)
# df_samp.loc[cr_om_index]

In [89]:
def critical_omission(df, seed):
    df_t = copy_df(df)
    cr_om_index = random_index(df_t.curHyp, 0.02, seed)
    dindex_dict["omission"]+=list(cr_om_index)
    result = df_t.loc[cr_om_index]
    random_ratio = [3/5, 2/3, 5/8, 5/7]
    np.random.seed(seed)
    split_sentences = result.curHyp.apply(lambda x: x.split(" "))
    len_element = split_sentences.apply(len)
    crop_element = len_element.apply(lambda x: int(x*(np.random.choice(random_ratio, 1))))
    start_idx = (len_element-crop_element).apply(lambda x: np.random.choice([i for i in range(x)], 1)[0])
    end_idx = start_idx+crop_element

    for i in cr_om_index:
        reduce_sentence = (" ".join([split_sentences[i][k] for k in range(start_idx[i], end_idx[i]+1)])).strip()
        cur_hyp = result.loc[i, "curHyp"]
        sidx = cur_hyp.find(reduce_sentence)

        conv_hyp = " ".join([split_sentences[i][j] for j in range(len_element[i]) if (start_idx[i] > j) or (end_idx[i] < j)])
        result.loc[i, "curHyp"] = conv_hyp
        result.loc[i, "tagLog"]["omission"] = sidx
        input_errors(result, i, "critical", sidx, reduce_sentence, "omission")

    del split_sentences, len_element, crop_element, start_idx, end_idx
    return convert_df(df, result)

df_crom = critical_omission(df_samp, 10)

In [90]:
df_crom.loc[cr_om_index].shape

(413, 7)

### Omission - Major

In [91]:
# np.random.seed(20)
# chv_index = random_index(df_crom.drop(index=dindex_dict["omission"]), 0.4, 20)
# omajor_tokens = df_crom.loc[chv_index].curHyp.apply(lambda x: kiwi.tokenize(x))
# nnp_info = omajor_tokens.apply(lambda x: find_tokens(x, "NNP"))
# change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
# ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 3]
# drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
# ch_srs = ch6.drop(index=drop_idx)

In [92]:
# # np.random.seed(20)
# change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
# ch6 = change_val[change_val.apply(lambda x: len(x[0])) > 3]
# # chv_index = random_index(ch6, 0.65, 20)
# drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
# ch_srs = ch6.drop(index=drop_idx)
# ch_srs

In [93]:
# ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1]    # 시리즈에 검출되는 고유명사가 있다면 drop 시킬 것!

In [94]:
def major_omission(df_local, seed, tag, slen, ratio):
    df_t = copy_df(df_local)
    np.random.seed(seed)
    nnp_info = kiwi_tokens.apply(lambda x: find_tokens(x, tag)).drop(index=dindex_dict["omission"], axis=0)
    np.random.seed(seed)
    change_val = nnp_info[nnp_info.apply(len) > 0].apply(lambda x: x[np.random.choice(len(x), 1)[0]])
    ch6 = change_val[change_val.apply(lambda x: len(x[0])) > slen]
    chv_index = random_index(ch6, ratio, seed)
    # drop_idx = ch6.loc[ch6.apply(lambda x: len(x[0].split("·")))>1].index.to_list()
    ch_srs = ch6.loc[chv_index]
    result = df_t.loc[chv_index]

    for i in chv_index:
        del_word, start, end = ch_srs.loc[i]
        cur_hyp = result.loc[i, "curHyp"]

        forward = cur_hyp[:start]
        backward = cur_hyp[start:]
        split_back = backward.split(" ")
        del_unit = split_back[0]
        back_mod = " ".join(split_back[1:])

        try:
            if start==0:
                conv_hyp = (forward+back_mod).strip(" ")
            elif cur_hyp[start-1] == '"' or cur_hyp[start-1]=='「' or cur_hyp[end]=="」" or cur_hyp[end]=='\"' :
                pattern = f'\"[^\"]*{del_word}[^\"]*\"|「[^」]*{del_word}[^」]*」'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            elif "(" in del_unit:
                pattern = f'{del_word}[^\)]+\)'
                find_comp = re.compile(pattern)
                find_res = find_comp.search(cur_hyp)
                start, end = find_res.span()
                while cur_hyp[end]!=" " and  end<len(cur_hyp):
                    end+=1
                while cur_hyp[start]!=" " and start>=0:
                    start-=1
                start+=1
                del_unit = cur_hyp[start:end]
                conv_hyp = (cur_hyp[:start].strip()+" "+cur_hyp[end:].strip()).strip()
            else:
                conv_hyp = (forward+back_mod).strip(" ")
            result.loc[i, "tagLog"]["omission"] = start
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "major", start, del_unit, "omission")
            dindex_dict["omission"].append(i)
        except:
            pass

    return convert_df(df_local, result)

df_maom = major_omission(df_crom, 20, "NNP", 3, 0.65)
df_mo = major_omission(df_maom, 36, "NNG", 2, 0.55)

In [95]:
df_maom.loc[df_maom.errors.apply(len)==0].shape[0]

18717

In [96]:
df_mo.loc[df_mo.errors.apply(len)==0].shape[0]

17635

In [98]:
df_mo.loc[df_mo.errors.apply(len)==1]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
547121,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the average monthly remuneration of each...,"예술인[「고용보험법」 제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술...","제77조의2제2항제2호 단서에 따른 단기예술인(이하 ""단기예술인""이라 한다)과 같은...","[{'severity': 'major', 'start_idx': 0, 'text':...",{'omission': 0}
547123,노동,고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률 시행령,Where the amount of monthly remuneration of ea...,"노무제공자[「고용보험법」 제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""...","제77조의6제2항제2호 단서에 따른 단기노무제공자(이하 ""단기노무제공자""라 한다)와...","[{'severity': 'major', 'start_idx': 0, 'text':...",{'omission': 0}
394206,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"Where the Minister of Trade, Industry and Ener...",산업통상자원부장관은 제4항이나 제5항에 따라 경제자유구역을 지정한 경우에는 대통령령...,산업통상자원부장관은 제4항이나 제5항에 따라 경제자유구역을 지정한 경우에는 대통령령...,"[{'severity': 'major', 'start_idx': 125, 'text...",{'omission': 125}
394231,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,Matters concerning the supply of exclusive lan...,"외국인투자기업, 국내복귀기업, 첨단기술 및 첨단제품 투자 기업, 핵심전략산업 투자 ...","외국인투자기업, 국내복귀기업, 첨단기술 및 첨단제품 투자 기업, 핵심전략산업 투자 ...","[{'severity': 'major', 'start_idx': 120, 'text...",{'omission': 120}
394245,통화ㆍ국채ㆍ금융,경제자유구역의 지정 및 운영에 관한 특별법,"The Minister of Trade, Industry and Energy may...",산업통상자원부장관은 경제자유구역개발계획을 변경할 수 있다. 시·도지사가 요청하는 경...,산업통상자원부장관은 경제자유구역개발계획을 같다.,"[{'severity': 'critical', 'start_idx': 23, 'te...",{'omission': 23}
...,...,...,...,...,...,...,...
194472,노동,산업재해보상보험법 시행령,"""Persons who are engaged in jobs prescribed by...","법 제125조제1항에서 ""대통령령으로 정하는 직종에 종사하는 자""란 다음 각 호의 ...",법 제125조제1항에서 다음 각 호의 어느 하나에 해당하는 자를 말한다.,"[{'severity': 'major', 'start_idx': 13, 'text'...",{'omission': 13}
416408,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률 시행령,Where a person performs an act falling under a...,신문·방송·잡지 등 불특정 다수인에 대한 광고를 통하여 법 제152조제2항 각 호의...,신문·방송·잡지 등 불특정 다수인에 대한 광고를 통하여 법 제152조제2항 각 호의...,"[{'severity': 'major', 'start_idx': 105, 'text...",{'omission': 105}
194518,노동,산업재해보상보험법 시행령,The Industrial Accident Compensation Insurance...,"이 영 시행 당시 종전의 규정에 따라 설치된 산업재해보상보험심의위원회, 정책전문위원...",이 영 시행 당시 종전의 규정에 따라 설치된 정책전문위원회 및 요양전문위원회는 각각...,"[{'severity': 'major', 'start_idx': 25, 'text'...",{'omission': 25}
194521,노동,산업재해보상보험법 시행령,The amended provisions of Article 68 (1) 1 (b)...,제68조제1항제1호나목의 개정규정은 이 영 시행 당시 요양 중인 사람부터 적용한다.,중인 사람부터 적용한다.,"[{'severity': 'critical', 'start_idx': 0, 'tex...",{'omission': 0}


## 3.2.Addition

###Addition-Major

In [99]:
tag = "NNG"
nouns =list(set(count_tokens.apply(lambda x: x[tag] if tag in x.keys() else []).sum()))
len(nouns)

3703

In [105]:
ma_ad_index = random_index(df_mo, 0.22, 55)
nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
nnpg_info.drop(index=drop_index, axis=0, inplace=True)
ma_ad_index = list(set(ma_ad_index)-set(drop_index))
np.random.seed(55)
add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
np.random.seed(55)
add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))

In [106]:
add_words

411178     부실
555763    모국어
80092      증표
84877     코스닥
81274       선
         ... 
82668      발병
406007    배당금
396112    적립식
77984     채권자
548401     실무
Name: KOR, Length: 4550, dtype: object

In [119]:
def major_addition(df, ratio, seed):
    np.random.seed(seed)
    df_t=copy_df(df)

    ma_ad_index = random_index(df_t, ratio, seed)
    nouns_lists = count_tokens[ma_ad_index].apply(lambda x: x["NNG"])
    nnpg_info = count_tokens[ma_ad_index].apply(lambda x: [val for val in list(set(x["NNG"])) if len(val)>1])
    drop_index = nnpg_info[nnpg_info.apply(len) == 0].index.to_list()
    nnpg_info.drop(index=drop_index, axis=0, inplace=True)
    ma_ad_index = list(set(ma_ad_index)-set(drop_index))
    np.random.seed(seed)
    add_position = nnpg_info.apply(lambda x: x[np.random.choice(range(0, len(x)))])
    np.random.seed(seed)
    add_words = nouns_lists.apply(lambda x: np.random.choice(list(set(nouns)-set(x))))
    result = df_t.loc[ma_ad_index]
    dindex_dict["addition"]+=ma_ad_index

    josa_list = ["의", "에 의한", "에", "만", "만의"]
    for i in ma_ad_index:
        cur_hyp = df.loc[i, "curHyp"]
        josa_add = np.random.choice(josa_list)
        word = add_position[i]
        start = cur_hyp.find(word)
        if word not in cur_hyp:
            continue
        add_word = add_words[i]+josa_add
        # print(start, cur_hyp)
        while start>=0 and cur_hyp[start]!=" ":
            start-=1
        conv_hyp = cur_hyp[:start+1].strip()+" "+f"{add_word}"+" "+cur_hyp[start+1:].strip()
        # print(add_word, word)
        # print(start, conv_hyp)
        result.loc[i, "tagLog"]["addition"] = add_word
        result.loc[i, "curHyp"] = conv_hyp
        input_errors(result, i, "major", start+1, add_word, "addition")

        try:
            o_index = result.loc[i, "tagLog"]["omission"]
            if o_index > start+1:
                result.loc[i, "tagLog"]["omission"] += (len(add_word)+1)
        except:
            pass
    return convert_df(df, result)

df_maad = major_addition(df_mo, 0.22, 55)

In [120]:
df_maad.loc[df_maad.errors.apply(len)>0].shape

(6972, 7)

## Mis-Translation

### 3.3.1.Mis-Translation : Minor

In [118]:
js_mstr_index = random_index(df_maad, 0.2, 1010)
js_tokens = df_maad.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))
jsd = {"가":"이", "을":"를", "로써":"으로써", "로서":"으로서", "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나", "와":"과"}
keys = list(jsd.keys())
for key in keys:
    jsd[jsd[key]] = key

tags=["JKS", "JKC", "JKO", "JKB", "JX", "JC"]
need_josa = None
for i, tag in enumerate(tags):
    if i==0:
        need_josa = js_tokens.apply(lambda x: find_tokens(x, tag))
    else:
        need_josa += js_tokens.apply(lambda x: find_tokens(x, tag))

drop_index = need_josa[need_josa.apply(len)==0].index
js_mstr_index = list(set(js_mstr_index)-set(drop_index))
need_josa.drop(index=drop_index, axis=0, inplace=True)
np.random.seed(1010)
trans_words = need_josa.apply(lambda x: x[np.random.choice(range(0,len(x)))])
trans_words

395580     (으로, 84, 86)
416588      (가, 39, 40)
399808      (를, 40, 41)
397909    (를, 152, 153)
547949      (을, 61, 62)
              ...      
399674     (에서, 29, 31)
189526      (은, 43, 44)
419406      (에, 11, 12)
401857      (에, 17, 18)
81056       (를, 10, 11)
Name: curHyp, Length: 4130, dtype: object

In [123]:
def josa_mistranslation(df, ratio, seed):
    df_t = copy_df(df)
    js_mstr_index = random_index(df_t, ratio, seed)
    js_tokens = df_maad.loc[js_mstr_index].curHyp.apply(lambda x: kiwi.tokenize(x))

    jsd = {"가":"이", "을":"를", "로써":"으로써", "로서":"으로서", "로":"으로", "로부터":"으로부터", "란":"이란", "나":"이나", "와":"과"}
    keys = list(jsd.keys())
    for key in keys:
        jsd[jsd[key]] = key

    tags=["JKS", "JKC", "JKO", "JKB", "JX", "JC"]
    need_josa = None
    for i, tag in enumerate(tags):
        if i==0:
            need_josa = js_tokens.apply(lambda x: find_tokens(x, tag))
        else:
            need_josa += js_tokens.apply(lambda x: find_tokens(x, tag))

    drop_index = need_josa[need_josa.apply(len)==0].index
    js_mstr_index = list(set(js_mstr_index)-set(drop_index))
    need_josa.drop(index=drop_index, axis=0, inplace=True)
    np.random.seed(seed)
    trans_words = need_josa.apply(lambda x: x[np.random.choice(range(0,len(x)))])
    result = df_t.loc[js_mstr_index]

    for i in js_mstr_index:
        cur_hyp = result.loc[i, "curHyp"]
        word, start, end = trans_words[i]
        try:
            conv_word = jsd[word]
            conv_hyp = cur_hyp[:start]+conv_word+cur_hyp[end:]
            result.loc[i, "tagLog"]["mis-translation"] = conv_word
            result.loc[i, "curHyp"] = conv_hyp
            input_errors(result, i, "minor", start, conv_word, "mis-translation")
            dindex_dict["mis-translation"].append(i)
        except:
            pass
        try:
            o_index = result.loc[i, "tagLog"]["omission"]
            if o_index > start:
                if len(word) > len(conv_word):
                    result.loc[i, "tagLog"]["omission"] -= (len(word)-len(conv_word))
                elif len(word) < len(conv_word):
                    result.loc[i, "tagLog"]["omission"] += (len(conv_word)-len(word))
        except:
            pass
    return convert_df(df, result)

df_jmt = josa_mistranslation(df_maad, 0.2, 1010)

In [124]:
temp = df_jmt.loc[js_mstr_index]
temp.loc[temp.tagLog.apply(len)==3]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
410759,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,"If necessary for protecting investors, the Fin...","금융위원회는 투자자 보호를 위하여 필요한 경우에는 공개매수자, 공개매수자의 특별관계...","금융위원회는 투자자 보호를 위하여 필요한 경우에는 표지에 의한 공개매수자, 공개매수...","[{'severity': 'major', 'start_idx': 49, 'text'...","{'omission': 56, 'addition': '표지에 의한', 'mis-tr..."
83241,통화ㆍ국채ㆍ금융,예금자보호법,Where deemed necessary to prepare for the inte...,"예금보험공사는 증권투자자보호기금, 보험보증기금, 신용관리기금의 출연금운용사업회계, ...","증권투자자보호기금, 보험보증기금, 신용관리기금의 출연금운용사업회계, 신용협동조합안전...","[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '미화에 의한', 'mis-tra..."
83486,통화ㆍ국채ㆍ금융,온라인투자연계금융업 및 이용자 보호에 관한 법률,Every online investment-linked financial busin...,온라인투자연계금융업자는 제1항 및 제2항에 따른 사항이 준수될 수 있도록 대통령령으...,온라인투자연계금융업자는 제1항 및 제2항에 따른 사항가 준수될 수 있도록 정하는 벤...,"[{'severity': 'major', 'start_idx': 41, 'text'...","{'omission': 41, 'addition': '벤처만의', 'mis-tran..."
411842,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,"With respect to special-purpose companies, the...",투자목적회사에 관하여는 이 법에 특별한 규정이 없으면 「상법」의 주식회사 또는 유한...,투자목적회사에 관하여는 이 법에 특별한 규정가 없으면 「상법」의 5의 주식회사 또는...,"[{'severity': 'major', 'start_idx': 44, 'text'...","{'omission': 47, 'addition': '5의', 'mis-transl..."
395693,통화ㆍ국채ㆍ금융,금융산업의 구조개선에 관한 법률,"The Korea Deposit Insurance Corporation may, w...",예금보험공사는 금융기관이 적기시정조치를 원활하게 이행할 수 있도록 하기 위하여 필요...,금융기관이 적기시정조치를 원활하게 이행할 수 있도록 하기 위하여 전복만 필요하다고 ...,"[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '전복만', 'mis-transl..."
84669,통화ㆍ국채ㆍ금융,인터넷전문은행 설립 및 운영에 관한 특례법,Where deemed that a retired executive officer ...,금융위원회(제14조제1항에 따라 조치를 하거나 같은 조 제2항에 따라 문책처분을 할...,금융위원회(제14조제1항에 따라 조치를 하거나 같은 조 제2항에 따라 문책처분을 할...,"[{'severity': 'major', 'start_idx': 176, 'text...","{'omission': 180, 'addition': '요금의', 'mis-tran..."
84859,통화ㆍ국채ㆍ금융,자산유동화에 관한 법률,Notwithstanding Article 462 of the Commercial ...,유동화전문회사는 상법 제583조의 규정에 의하여 준용되는 동법 제462조의 규정에 ...,상법 제583조의 규정에 의하여 준용되는 동법 제462조의 규정에 불구하고 정관이 ...,"[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '차액에 의한', 'mis-tra..."
85020,통화ㆍ국채ㆍ금융,전자단기사채등의 발행 및 유통에 관한 법률,Where the bankruptcy or dissolution of an acco...,"한국예탁결제원은 계좌관리기관의 파산·해산, 그 밖에 대통령령으로 정하는 사유가 발생...","계좌관리기관의 파산·해산, 그 밖에 대통령령으로 정하는 사유가 등기에 의한 발생한 ...","[{'severity': 'major', 'start_idx': 0, 'text':...","{'omission': 0, 'addition': '등기에 의한', 'mis-tra..."
397375,통화ㆍ국채ㆍ금융,금융지주회사법,Standards for imposing penalty surcharges unde...,제1항의 규정에 의한 과징금의 부과기준 기타 과징금의 부과에 관하여 필요한 사항은 ...,제1항의 규정에 의한 부과기준 기타 과징금의 부과에 관하여 필요한 생활환경에 의한 ...,"[{'severity': 'major', 'start_idx': 12, 'text'...","{'omission': 12, 'addition': '생활환경에 의한', 'mis-..."
413942,통화ㆍ국채ㆍ금융,자본시장과 금융투자업에 관한 법률,A person who fails to comply with the matters ...,제117조의15에 따라 투자자 피해가 발생하지 아니하도록 하기 위한 사항을 이행하지...,제117조의15에 따라 피해가 최소의 발생하지 아니하도록 하기 위한 사항를 이행하지...,"[{'severity': 'major', 'start_idx': 13, 'text'...","{'omission': 13, 'addition': '최소의', 'mis-trans..."


## 나중에

In [ ]:
pat = r"\"[ㄱ-ㅎ|가-힣]+\""
quote_comp = re.compile(pat)
quote_words = df_samp.loc[df_samp.KOR.str.contains(pat)].KOR.apply(lambda x: quote_comp.findall(x))
quote_words

547121                    ["단기예술인"]
547123                  ["단기노무제공자"]
394156                   ["경제자유구역"]
394158                      ["외국인"]
394159                  ["외국인투자기업"]
                    ...            
194419    ["위원장", "재심사위원회", "소위원회"]
194455             ["진찰", "특진의료기관"]
416399        ["의결권권유자", "의결권피권유자"]
194470             ["근로자", "진폐근로자"]
194496         ["근로자", "특수형태근로종사자"]
Name: KOR, Length: 1716, dtype: object

In [ ]:
quword4 = quote_words[quote_words.apply(lambda x: max(list(map(lambda k: len(k), x)))) > 8]

In [ ]:
qw_srs = quword4.apply(lambda x: sorted(x, key=lambda k: len(k), reverse=True)[0].strip('"').rstrip("등"))
sort_index = qw_srs.apply(len).sort_values().index
qw_srs[sort_index]

548555                출산전후급여
81365                 대부중개업자
397510                은행지주회사
207755                지정의료기관
83487                 여신금융기관
                 ...        
422493          주택담보노후연금보증채무
418500         금융투자상품거래청산업인가
416776        외국금융투자상품거래청산회사
556053        산업재해보상보험재심사위원회
519095    부당이득징수금체납정보공개심의위원회
Name: KOR, Length: 409, dtype: object

In [ ]:
origin_text = list(set(list(qw_srs.values)))
origin_text
# origin_text

## 다른 오류 생성

In [ ]:
df_samp.loc[df_samp.errors.apply(len)==0].shape[0]

3903

In [ ]:
not_yet_index = df_samp.loc[df_samp.errors.apply(len)==0].index

In [ ]:
temp = df_samp.loc[not_yet_index]
temp.loc[temp.curHyp.str.contains("할 수 있다")].iloc[:, 2:4]

In [ ]:
pattern = r"[\d]+조[^\항]*[\d]+항"
hangho = re.compile(pattern)
txtx = "집합투자업자는 제80조제4항 및 제86조제1항에도 불구하고 법 제81조제1항 단서 및 제234조제4항에 따라 상장지수집합투자기구(투자자 보호 등을 고려하여 금융위원회가 정하여 고시하는 상장지수집합투자기구에 한정한다)의 집합투자재산을 다음 각 호의 방법으로 운용할 수 있다."
sres = hangho.search(txtx)
print(sres)
fnum = txtx[sres.span()[0]]
snum = txtx[sres.span()[1]-2]
print(fnum, snum)

<re.Match object; span=(9, 15), match='80조제4항'>
8 4


In [ ]:
def make_dict(df):
    output_json = dict()
    output_json["data"] = []
    for i in df.index:
        base_form = {"category": None, "en": None, "errs": None, "hyp": None, "ko": None, "law_name": None}
        if len(df.loc[i, "errors"])==0:
            continue
        base_form["category"] = df.loc[i, "category"]
        base_form["en"] = df.loc[i, "ENG"]
        base_form["errs"] = df.loc[i, "errors"]
        base_form["hyp"] = df.loc[i, "curHyp"]
        base_form["ko"] = df.loc[i, "KOR"]
        base_form["law_name"] = df.loc[i, "lawName"]
        output_json["data"].append(base_form)
    return output_json

rrrr = make_dict(df_samp)

In [ ]:
with open(os.path.join(gdrive_path, "data_0823.json"), "w") as f:
  json.dump(rrrr, f, ensure_ascii=False, indent=4)

In [125]:
def convert_index(df_new):
    df = copy_df(df_new)
    for i in df.index:
        cur_hyp = df.loc[i, "curHyp"]
        for j, val in enumerate(df.loc[i, "errors"]):
            e_case = val["type"]
            if e_case=="omission":
                df.loc[i, "errors"][j]["start_idx"] = df.loc[i, "tagLog"][e_case]
            elif e_case=="mis-translation":
                continue
            else:
                idx = cur_hyp.find(df.loc[i, "tagLog"][e_case])
                df.loc[i, "errors"][j]["start_idx"] = idx
    return df

temp = convert_index(df_jmt)
temp.loc[[402202, 402265]]


,category,lawName,ENG,KOR,curHyp,errors,tagLog
402202,통화ㆍ국채ㆍ금융,보험업법 시행령,In other cases similar to subparagraphs 1 and ...,그 밖에 제1호 및 제2호에 준하는 경우로서 금융위원회가 정하여 고시하는 경우,그 밖에 제1호 및 제2호에 준하는 경우로서 금융정하여 인만 고시하는 경우,"[{'severity': 'major', 'start_idx': 27, 'text'...","{'omission': 27, 'addition': '인만'}"
402265,통화ㆍ국채ㆍ금융,보험업법 시행령,Where any contribution money has not been paid...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,제5항에 따른 이용만 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 ...,"[{'severity': 'major', 'start_idx': 81, 'text'...","{'omission': 81, 'addition': '이용만', 'mis-trans..."


In [126]:
df_jmt.loc[[402202, 402265]]

,category,lawName,ENG,KOR,curHyp,errors,tagLog
402202,통화ㆍ국채ㆍ금융,보험업법 시행령,In other cases similar to subparagraphs 1 and ...,그 밖에 제1호 및 제2호에 준하는 경우로서 금융위원회가 정하여 고시하는 경우,그 밖에 제1호 및 제2호에 준하는 경우로서 금융정하여 인만 고시하는 경우,"[{'severity': 'major', 'start_idx': 27, 'text'...","{'omission': 27, 'addition': '인만'}"
402265,통화ㆍ국채ㆍ금융,보험업법 시행령,Where any contribution money has not been paid...,제5항에 따른 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 대하여 ...,제5항에 따른 이용만 납부기한까지 출연금을 내지 아니한 경우에는 내야 할 출연금에 ...,"[{'severity': 'major', 'start_idx': 77, 'text'...","{'omission': 81, 'addition': '이용만', 'mis-trans..."
